In [243]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import numpy as np

In [31]:
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1')
html = BeautifulSoup(url.text)

In [79]:
def make_table(html):
    table = html.find('table', class_ = 'type_2')
    table = pd.read_html(str(table))[0]
    del table['N']
    del table['토론실']
    table = table.dropna(subset='종목명', ignore_index=True)
    return table

In [117]:
kospi_page = int(html.find('td', class_ = 'pgRR').find('a').get('href').split('=')[-1])

In [119]:
# 퀴즈 : 모든 코스피 페이지를 가져와서 하나의 데이터 프레임 만들기

In [123]:
total = []

for i in range(1, kospi_page+1):
    url = requests.get(f'https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={i}')
    html = BeautifulSoup(url.text)
    time.sleep(0.5)
    total.append(make_table(html))

In [147]:
kospi = pd.concat([total[i] for i in range(0, kospi_page)], ignore_index=True)
kospi['소속'] = 'KOSPI'
kospi.to_excel('kospi.xlsx', index = False)
kospi

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,삼성전자,57500.0,상승 200,+0.35%,100.0,3432625.0,5969783.0,50.03,6034136.0,11.62,9.03,KOSPI
1,SK하이닉스,199400.0,"하락 5,600",-2.73%,5000.0,1451637.0,728002.0,55.95,1305154.0,13.92,-15.61,KOSPI
2,LG에너지솔루션,377500.0,"하락 8,000",-2.08%,500.0,883350.0,234000.0,4.54,61459.0,-243.55,6.36,KOSPI
3,삼성바이오로직스,1196000.0,"상승 45,000",+3.91%,2500.0,851241.0,71174.0,13.54,59637.0,80.88,9.12,KOSPI
4,현대차,203500.0,"하락 3,500",-1.69%,5000.0,426162.0,209416.0,37.41,226335.0,4.45,13.68,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...
2305,TIGER 200 산업재,8245.0,하락 35,-0.42%,0.0,20.0,240.0,0.00,10771.0,NaN,NaN,KOSPI
2306,PLUS 코스닥150선물인버스,5480.0,상승 35,+0.64%,0.0,19.0,350.0,0.00,10352.0,NaN,NaN,KOSPI
2307,HANARO 글로벌백신치료제MSCI,11800.0,보합0,0.00%,0.0,18.0,150.0,0.00,79.0,NaN,NaN,KOSPI
2308,HANARO 미국애그테크,11705.0,상승 15,+0.13%,0.0,18.0,150.0,0.00,19.0,NaN,NaN,KOSPI


In [151]:
# 퀴즈 : 코스닥까지 가져와서 코스피 + 코스닥을 합친 하나의 데이터 프레임 만들기 per 값이 0보다 크고 10보다 작은 행만 추출

In [153]:
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1')
html = BeautifulSoup(url.text)

In [159]:
kosdaq_page = int(html.find('td', class_ = 'pgRR').find('a').get('href').split('=')[-1])

In [171]:
kosdaq_total = []

for i in tqdm(range(1, kosdaq_page+1)):
    url = requests.get(f'https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={i}')
    html = BeautifulSoup(url.text)
    time.sleep(0.5 + random.random())
    kosdaq_total.append(make_table(html))

100%|███████████████████████████████████████████| 36/36 [00:51<00:00,  1.43s/it]


In [173]:
kosdaq = pd.concat([kosdaq_total[i] for i in range(0, kosdaq_page)], ignore_index=True)
kosdaq['소속'] = 'KOSDAQ'

kosdaq

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,알테오젠,393000.0,"상승 10,000",+2.61%,500.0,209543.0,53319.0,15.82,200223.0,-7145.45,-2.27,KOSDAQ
1,에코프로비엠,135400.0,"하락 5,400",-3.84%,500.0,132423.0,97801.0,10.67,391816.0,-60.94,-0.64,KOSDAQ
2,HLB,88500.0,상승 500,+0.57%,500.0,116278.0,131387.0,22.57,666132.0,-62.15,-32.55,KOSDAQ
3,에코프로,66400.0,"하락 2,200",-3.21%,100.0,88404.0,133138.0,20.02,601265.0,-48.64,3.43,KOSDAQ
4,레인보우로보틱스,390500.0,상승 500,+0.13%,500.0,75756.0,19400.0,8.28,189194.0,-370.14,-0.93,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...
1786,투비소프트,632.0,상승 2,+0.32%,500.0,63.0,9900.0,2.26,426655.0,-0.20,-41.73,KOSDAQ
1787,셀리버리,152.0,"하락 6,528",-97.72%,500.0,57.0,37183.0,2.30,16946754.0,-0.92,472.60,KOSDAQ
1788,CNH,109.0,보합0,0.00%,2500.0,41.0,37200.0,6.07,0.0,-0.04,-18.80,KOSDAQ
1789,소프트센우,7050.0,상승 140,+2.03%,200.0,25.0,357.0,0.02,107.0,-110.16,NaN,KOSDAQ


In [175]:
df = pd.concat([kospi, kosdaq], ignore_index=True)
df

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,삼성전자,57500.0,상승 200,+0.35%,100.0,3432625.0,5969783.0,50.03,6034136.0,11.62,9.03,KOSPI
1,SK하이닉스,199400.0,"하락 5,600",-2.73%,5000.0,1451637.0,728002.0,55.95,1305154.0,13.92,-15.61,KOSPI
2,LG에너지솔루션,377500.0,"하락 8,000",-2.08%,500.0,883350.0,234000.0,4.54,61459.0,-243.55,6.36,KOSPI
3,삼성바이오로직스,1196000.0,"상승 45,000",+3.91%,2500.0,851241.0,71174.0,13.54,59637.0,80.88,9.12,KOSPI
4,현대차,203500.0,"하락 3,500",-1.69%,5000.0,426162.0,209416.0,37.41,226335.0,4.45,13.68,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...
4096,투비소프트,632.0,상승 2,+0.32%,500.0,63.0,9900.0,2.26,426655.0,-0.20,-41.73,KOSDAQ
4097,셀리버리,152.0,"하락 6,528",-97.72%,500.0,57.0,37183.0,2.30,16946754.0,-0.92,472.60,KOSDAQ
4098,CNH,109.0,보합0,0.00%,2500.0,41.0,37200.0,6.07,0.0,-0.04,-18.80,KOSDAQ
4099,소프트센우,7050.0,상승 140,+2.03%,200.0,25.0,357.0,0.02,107.0,-110.16,NaN,KOSDAQ


In [181]:
df[(0 < df['PER']) & (df['PER'] < 10)].reset_index()

,index,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,4,현대차,203500.0,"하락 3,500",-1.69%,5000.0,426162.0,209416.0,37.41,226335.0,4.45,13.68,KOSPI
1,5,삼성전자우,48200.0,상승 200,+0.42%,100.0,396631.0,822887.0,73.79,365845.0,9.74,NaN,KOSPI
2,7,기아,93900.0,"하락 1,000",-1.05%,5000.0,373415.0,397673.0,39.40,326957.0,3.90,20.44,KOSPI
3,9,KB금융,81500.0,하락 500,-0.61%,5000.0,320726.0,393528.0,75.62,260821.0,7.04,8.44,KOSPI
4,13,현대모비스,255500.0,"하락 1,000",-0.39%,5000.0,237602.0,92995.0,41.68,38558.0,6.97,8.73,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,3942,케이피티유,3495.0,하락 20,-0.57%,500.0,200.0,5726.0,0.28,702.0,9.55,6.55,KOSDAQ
619,3949,다산솔루에타,1108.0,하락 1,-0.09%,500.0,199.0,17950.0,0.54,4546.0,0.64,55.28,KOSDAQ
620,3973,디모아,3550.0,하락 5,-0.14%,500.0,175.0,4942.0,1.03,1466.0,3.08,-4.88,KOSDAQ
621,3979,씨엑스아이,57.0,상승 1,+1.79%,0.0,171.0,300578.0,0.00,609387.0,0.55,6.57,KOSDAQ


In [371]:
dic = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"}
url = requests.get('https://finance.naver.com/item/sise_day.naver?code=005930&page=1', headers = dic)
html = BeautifulSoup(url.text)

In [211]:
table = html.find('table')
table = pd.read_html(str(table))[0].dropna()
table

,날짜,종가,전일비,시가,고가,저가,거래량
1,2025.02.25,57400.0,상승 100,56600.0,57800.0,56500.0,8285317.0
2,2025.02.24,57300.0,하락 900,57300.0,57700.0,57200.0,14138471.0
3,2025.02.21,58200.0,하락 200,58400.0,58500.0,57100.0,22198428.0
4,2025.02.20,58400.0,하락 300,59100.0,59100.0,58100.0,21589059.0
5,2025.02.19,58700.0,"상승 1,800",57400.0,58900.0,57300.0,25395151.0
9,2025.02.18,56900.0,상승 900,56200.0,57200.0,55900.0,22131007.0
10,2025.02.17,56000.0,보합0,56100.0,56500.0,55700.0,11916027.0
11,2025.02.14,56000.0,상승 200,56000.0,57300.0,56000.0,23979779.0
12,2025.02.13,55800.0,보합0,56100.0,56400.0,55600.0,22448377.0
13,2025.02.12,55800.0,상승 100,55100.0,55900.0,54500.0,26428596.0


In [237]:
url = requests.get('https://finance.naver.com/sise/dividend_list.naver?&page=1')
html = BeautifulSoup(url.text)

In [283]:
# 3년전 < 2년전 < 1년전 < 배당금 발굴하기

def get_table(html):
    table = html.find('table', class_ = 'type_1 tb_ty')
    table = pd.read_html(str(table), header = 1)[0]
    table = table.dropna(subset = '종목명')
    return table

# table['3년전'] = table['3년전'].str.replace('-', '0').astype('int32')
# table['2년전'] = table['2년전'].str.replace('-', '0').astype('int32')

In [287]:
allocation_max_page = int(html.find('td', class_ = 'pgRR').find('a').get('href').split('=')[-1])

In [289]:
allocation_table = []

for i in tqdm(range(1, allocation_max_page + 1)):
    time.sleep(0.5 + random.random())
    url = requests.get(f'https://finance.naver.com/sise/dividend_list.naver?&page={i}')
    html = BeautifulSoup(url.text)
    allocation_table.append(get_table(html))

100%|███████████████████████████████████████████| 17/17 [00:21<00:00,  1.24s/it]


In [404]:
df2 = pd.concat([data for data in allocation_table], ignore_index=True)

In [406]:
df2['3년전'] = df2['3년전'].str.replace('-', '0').fillna(0).astype('int32')
df2['2년전'] = df2['2년전'].str.replace('-', '0').fillna(0).astype('int32')

In [408]:
df2

,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전
0,이지스밸류리츠,4375.0,24.12,823.0,18.81,-1361.40,-0.57,-116.13,0.66,406.0,293,292
1,제이알글로벌리츠,2510.0,24.06,390.0,15.54,158.83,2.82,32.42,0.94,380.0,366,330
2,레드캡투어,14090.0,25.03,2150.0,15.26,177.53,-,-,-,450.0,450,400
3,NH프라임리츠,4540.0,24.11,658.0,14.49,137.10,6.38,11.26,0.72,531.0,225,248
4,신한알파리츠,6050.0,24.09,806.0,13.32,547.91,0.61,183.31,1.19,365.0,367,321
...,...,...,...,...,...,...,...,...,...,...,...,...
840,포스코엠텍,13980.0,25.03,20.0,0.14,146.62,-,-,-,35.0,0,80
841,삼천당제약,185900.0,24.12,200.0,0.11,-,-,-,-,0.0,0,0
842,솔루스첨단소재,10130.0,24.12,10.0,0.10,-,-,-,-,50.0,0,48
843,대주전자재료,114800.0,24.12,100.0,0.09,-,-,-,-,0.0,0,100


In [410]:
df2[(df2['배당금'] > df2['1년전']) & (df2['1년전'] > df2['2년전']) & (df2['2년전'] > df2['3년전'])].reset_index().head(5)

,index,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전
0,0,이지스밸류리츠,4375.0,24.12,823.0,18.81,-1361.40,-0.57,-116.13,0.66,406.0,293,292
1,1,제이알글로벌리츠,2510.0,24.06,390.0,15.54,158.83,2.82,32.42,0.94,380.0,366,330
2,17,현대해상,24200.0,24.03,2063.0,8.52,26.61,7.81,4.56,0.40,1965.0,1480,1000
3,24,도이치모터스,4735.0,24.12,380.0,8.02,-,-,-,-,370.0,360,350
4,32,HS애드,7210.0,25.03,550.0,7.63,39.29,-,-,-,450.0,400,350


In [402]:
from datetime import datetime
today = str(datetime.today().date())

text = f"{today}기준 3년 연속 배당금이 상승한 기업 리스트\n\n"

for i in range(len(df)):
    row = df.iloc[i]
    name = row['종목명']
    a, b, c ,d = row['배당금'], row['1년전'], row['2년전'], row['3년전']
    text += f'{name} : {int(d):,}원 -> {int(c):,}원 -> {int(b):,}원 -> {int(a):,}원 \n'

print(text)
f = open('3년 연속 배당금이 상승한 기업.txt', 'wt')
f.write(text)
f.close()

2025-02-25기준 3년 연속 배당금이 상승한 기업 리스트

이지스밸류리츠 : 292원 -> 293원 -> 406원 -> 823원 
제이알글로벌리츠 : 330원 -> 366원 -> 380원 -> 390원 
레드캡투어 : 400원 -> 450원 -> 450원 -> 2,150원 
NH프라임리츠 : 248원 -> 225원 -> 531원 -> 658원 
신한알파리츠 : 321원 -> 367원 -> 365원 -> 806원 
이크레더블 : 2,720원 -> 1,040원 -> 780원 -> 1,590원 
미래에셋맵스리츠 : 268원 -> 279원 -> 284원 -> 284원 
서호전기 : 1,000원 -> 1,500원 -> 1,500원 -> 2,500원 
HB인베스트먼트 : 0원 -> 0원 -> 0원 -> 200원 
KB스타리츠 : 0원 -> 0원 -> 353원 -> 376원 
크레버스 : 2,000원 -> 1,800원 -> 2,000원 -> 1,500원 
NH올원리츠 : 106원 -> 316원 -> 288원 -> 340원 
코람코라이프인프라리츠 : 475원 -> 350원 -> 366원 -> 416원 
이리츠코크렙 : 354원 -> 414원 -> 395원 -> 390원 
미래에셋글로벌리츠 : 0원 -> 180원 -> 288원 -> 252원 
정상제이엘에스 : 530원 -> 530원 -> 530원 -> 530원 
케이카 : 750원 -> 760원 -> 760원 -> 1,150원 
현대해상 : 1,000원 -> 1,480원 -> 1,965원 -> 2,063원 
코리아에셋투자증권 : 450원 -> 700원 -> 200원 -> 500원 
영보화학 : 100원 -> 50원 -> 50원 -> 350원 
디앤디플랫폼리츠 : 78원 -> 291원 -> 281원 -> 268원 
마스턴프리미어리츠 : 0원 -> 109원 -> 299원 -> 127원 
무학 : 230원 -> 230원 -> 230원 -> 520원 
HRS : 200원 -> 200원 -> 200원 -> 400원 
도이치모터스